In [31]:
#  import necessary libraries
!pip install kaggle
!pip install pandas
!pip install -U scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 60.4/60.4 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
    --------------------------------------- 0.2/9.3 MB 6.3 MB/s eta 0:00:02
   - -------------------------------------- 0.4/9.3 MB 3.8 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/9.3 MB 5.5 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/9.3 MB 6.1 MB/s eta 0:00:02
   ----- ---------------------------------- 1.3/9.3 MB 5.9 MB/s eta 0:00:02
   ------- -------------------------------- 1.7/9.3 MB 5.9 MB/s eta 0:00:02
   -------- ------------------------------- 1.9/9.3 MB 6.2 MB/s eta 0:00:02
   --------- ------------------------------ 2.2/9.3 MB 6.1 MB/s eta 0:00:02
   ---------- ----------------------------- 2.5/9.3 MB 5.8 MB/s eta 0:00:02
   ----------- --

In [40]:
#  import of libraries

import numpy as np 
import pandas as pd 
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# WORKING WITH DATA SECTION :

In [12]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd() # set the path of the kaggle.json file here

competition_name = 'playground-series-s3e24'

!kaggle competitions download -c $competition_name


  0%|          | 0.00/6.81M [00:00<?, ?B/s]
 15%|█▍        | 1.00M/6.81M [00:01<00:06, 917kB/s]
 29%|██▉       | 2.00M/6.81M [00:01<00:03, 1.52MB/s]
 44%|████▍     | 3.00M/6.81M [00:01<00:02, 1.97MB/s]
 59%|█████▊    | 4.00M/6.81M [00:02<00:01, 2.18MB/s]
 73%|███████▎  | 5.00M/6.81M [00:02<00:00, 2.43MB/s]
 88%|████████▊ | 6.00M/6.81M [00:02<00:00, 2.61MB/s]
100%|██████████| 6.81M/6.81M [00:03<00:00, 2.73MB/s]
100%|██████████| 6.81M/6.81M [00:03<00:00, 2.21MB/s]


In [16]:
# check the files in the input folder

for dirname, _, filenames in os.walk('/dataset_Kaggel'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [23]:
FILE_PATH = './dataset_Kaggel/init/'
OUTPUT_DIR = './dataset_Kaggel/working_result/'

train = pd.read_csv(FILE_PATH + 'train.csv')
test = pd.read_csv(FILE_PATH + 'test.csv')

train_file_path = FILE_PATH + 'train.csv'
train_data = pd.read_csv(train_file_path)

test_file_path = FILE_PATH + 'test.csv'
test_data = pd.read_csv(test_file_path)

In [29]:
# Display the first few rows of the training data and summary information
print("Training Data : \n",train_data.head(),"\n")
print("Training Data Description : \n",train_data.describe(),"\n")
print("Testing Data Description : \n",test_data.describe())

Training Data : 
    id  age  height(cm)  weight(kg)  waist(cm)  eyesight(left)  \
0   0   55         165          60       81.0             0.5   
1   1   70         165          65       89.0             0.6   
2   2   20         170          75       81.0             0.4   
3   3   35         180          95      105.0             1.5   
4   4   30         165          60       80.5             1.5   

   eyesight(right)  hearing(left)  hearing(right)  systolic  ...  HDL  LDL  \
0              0.6              1               1       135  ...   40   75   
1              0.7              2               2       146  ...   57  126   
2              0.5              1               1       118  ...   45   93   
3              1.2              1               1       131  ...   38  102   
4              1.0              1               1       121  ...   44   93   

   hemoglobin  Urine protein  serum creatinine  AST  ALT  Gtp  dental caries  \
0        16.5              1              

# MODEL TRAINING SECTION :

Spiliting data to training and testing

In [33]:
X = train_data.drop(['id', 'smoking'], axis=1)
y = train_data['smoking']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)


#  Print out the shapes of the scaled training and validation sets to confirm their dimensions
print(X_train_scaled.shape, X_val_scaled.shape, y_train.shape, y_val.shape)

(127404, 22) (31852, 22) (127404,) (31852,)


Training data using Logistic Regression to test accuracy

In [43]:
LR_model = LogisticRegression(max_iter=1000, random_state=42)

LR_model.fit(X_train_scaled, y_train)

# Predict probabilities on the validation set
y_val_prob_LR = LR_model.predict_proba(X_val_scaled)[:, 1]

# Calculate ROC AUC score
roc_auc_LR = roc_auc_score(y_val, y_val_prob_LR)
print('ROC AUC score for Logistic Regression: ', roc_auc_LR)

ROC AUC score for Logistic Regression:  0.8319413944561205


Training data using KNeighborsClassifier to test accuracy

In [44]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train,y_train)
y_val_prob_KNN = knn_model.predict_proba(X_val)[:, 1]

# Calculate ROC AUC score
roc_auc_KNN = roc_auc_score(y_val, y_val_prob_KNN)
print('ROC AUC score for KNN: ', roc_auc_KNN)

ROC AUC score for KNN:  0.7585883972441365


Training data using Decision Tree to test accuracy

In [46]:
DT_model = DecisionTreeClassifier()
DT_model.fit(X_train, y_train)
y_val_prob_DT = DT_model.predict(X_val)

# Calculate ROC AUC score
roc_auc_DT = roc_auc_score(y_val, y_val_prob_DT)
print('ROC AUC score for Decision Tree: ', roc_auc_DT)

ROC AUC score for Decision Tree:  0.6883906123508766


As we can see best accuracy was in Logistic Regression model so it will b our model 😉

In [47]:
X_test = test_data.drop('id', axis=1)
X_test_scaled = scaler.transform(X_test)

# Predict probabilities on the test set
test_probabilities = LR_model.predict_proba(X_test_scaled)[:, 1]

# Prepare the submission file
result_kaggle_submit = pd.DataFrame({'id': test_data['id'], 'smoking': test_probabilities})
result_kaggle_submit.head()

print(result_kaggle_submit.head())

       id   smoking
0  159256  0.394570
1  159257  0.197513
2  159258  0.639809
3  159259  0.126900
4  159260  0.653465


In [49]:
result_kaggle_submit.to_csv(OUTPUT_DIR + 'submission.csv', index=False)

In [54]:
!kaggle competitions submit -c playground-series-s3e24 -f ./dataset_Kaggel/working_result/submission.csv -m "My submission it is in the bag"

Successfully submitted to Binary Prediction of Smoker Status using Bio-Signals



  0%|          | 0.00/2.77M [00:00<?, ?B/s]
  1%|          | 16.0k/2.77M [00:00<00:20, 143kB/s]
 14%|█▎        | 384k/2.77M [00:00<00:01, 1.98MB/s]
 68%|██████▊   | 1.89M/2.77M [00:00<00:00, 7.67MB/s]
 96%|█████████▌| 2.66M/2.77M [00:00<00:00, 2.84MB/s]
100%|██████████| 2.77M/2.77M [00:01<00:00, 1.67MB/s]
